In [2]:
import pandas as pd
import csv

data = pd.read_excel("/content/Technology Skills.xlsx")

In [3]:
data.head()

,O*NET-SOC Code,Title,Example,Commodity Code,Commodity Title,Hot Technology
0,11-1011.00,Chief Executives,Adobe Systems Adobe Acrobat,43232202,Document management software,Y
1,11-1011.00,Chief Executives,AdSense Tracker,43232306,Data base user interface and query software,N
2,11-1011.00,Chief Executives,Atlassian JIRA,43232201,Content workflow software,Y
3,11-1011.00,Chief Executives,Blackbaud The Raiser's Edge,43232303,Customer relationship management CRM software,Y
4,11-1011.00,Chief Executives,ComputerEase Construction Accounting,43231601,Accounting software,N


In [4]:
skill_data = data['Example'].unique()
skils_list = list(skill_data.flatten())
len(skils_list)

8824

In [5]:
data_frame = pd.read_csv('/content/dice_com-job_us_sample.csv',error_bad_lines=False, engine="python")
data_frame.head()

Skipping line 660: unexpected end of data


,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,NaN,SEE BELOW,418ff92580b270ef4e7c14f0ddfc36b4
1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",Information Security Engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9
2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",Business Solutions Architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Enterprise Solutions Architecture, business in...",46baa1f69ac07779274bcd90b85d9a72
3,https://www.dice.com/jobs/detail/Java-Develope...,TransTech LLC,Full Time,Java DeveloperFull-time/direct-hireBolingbrook...,Dice Id : 10113627,"Bolingbrook, IL","Java Developer (mid level)- FT- GREAT culture,...",2 weeks ago,Telecommuting not available|Travel not required,NaN,Please see job description,3941b2f206ae0f900c4fba4ac0b18719
4,https://www.dice.com/jobs/detail/DevOps-Engine...,Matrix Resources,Full Time,Midtown based high tech firm has an immediate ...,Dice Id : matrixga,"Atlanta, GA",DevOps Engineer,48 minutes ago,Telecommuting not available|Travel not required,NaN,"Configuration Management, Developer, Linux, Ma...",45efa1f6bc65acc32bbbb953a1ed13b7


In [6]:
job_description = data_frame['jobdescription']
job_description = list(job_description)
len(job_description)

658

In [7]:
!pip install spacy==3.0.6

     |████████████████████████████████| 12.8 MB 17.7 MB/s 
     |████████████████████████████████| 451 kB 47.9 MB/s 
     |████████████████████████████████| 9.1 MB 59.6 MB/s 
     |████████████████████████████████| 628 kB 50.6 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [8]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
# Load pre-existing spacy model
nlp_1 = spacy.load('en_core_web_sm')
# Getting the pipeline component
ner = nlp_1.get_pipe("ner")

In [10]:
corpus = []

for data in job_description[0:100]:
  doc = nlp_1(data)
  for sent in doc.sents:
      corpus.append(sent.text)

In [11]:
nlp = spacy.blank("en")
ruler = nlp.add_pipe("entity_ruler")
count = 1

for skil in skils_list[0:1000]:
    skil_split = skil.split()
    pattern = []
    for i in skil_split:
      pattern.append({'lower': i.lower()})  
    ruler.add_patterns([{"label": "skill", "pattern": pattern}])

In [12]:
TRAIN_DATA_TEXT = []

for sentence in corpus:
    doc = nlp(sentence)
    entities = []
    
    for ent in doc.ents:
        entities.append([ent.start_char, ent.end_char, ent.label_])
        
    if len(entities)!=0:
      TRAIN_DATA_TEXT.append((sentence, {"entities": entities}))
    
print(TRAIN_DATA_TEXT)

[('Looking for Selenium engineers...must have solid java coding skills\xa0', {'entities': [[12, 20, 'skill']]}), ('No associates report to the person in this role on a permanent basis • Analyze and resolve complex quality issues critical to solution delivery • Fast-paced environment of freedom and responsibility PREFERRED QUALIFICATIONS • Hands on Java Development experience • Proficient in Ab Initio ETL and Batch testing • Hands-on knowledge of Soap UI, ITKO Lisa, and Selenium •', {'entities': [[359, 367, 'skill']]}), ('Hands on experience with Unix, KSH, and Bourne Shell Scripting •', {'entities': [[25, 29, 'skill']]}), ('eCommerce/Retail QA experience • Excellent written and verbal communication skills • Self-motivated with a strong work ethic and a positive attitude • Desire to work in a fast-paced, results orientated team • Experience with a scripting language such as Perl • Passion to understand, learn, dissect and improve new technologies • Proficient in one or more of the follo

In [13]:
import pickle

In [14]:
with open('/content/Training_data/train_data.txt', 'wb') as f:
    pickle.dump(TRAIN_DATA_TEXT, f)

In [15]:
nlp=spacy.load('en_core_web_sm')

# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [16]:
with open('/content/Training_data/train_data.txt', 'rb') as f:
    TRAIN_DATA = pickle.load(f)

In [17]:
# Adding labels to the `ner`
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [18]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [19]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training.example import Example

In [20]:
# 500
# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(10):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(10, 20, 1.5))
    # for batch in batches:
    #     texts, annotations = zip(*batch)
    #     nlp.update(
    #                 texts,  # batch of texts
    #                 annotations,  # batch of annotations
    #                 drop=0.5,  # dropout - make it harder to memorise data
    #                 losses=losses,
    #             )
    #     print("Losses", losses)


    for batch in batches:
        texts, annotations = zip(*batch)

        example = []
      # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
#             print(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))

            # Update the model
            nlp.update(example, drop=0.1, losses=losses)

            print("Losses", losses)

Losses {'ner': 13.322802696689966}
Losses {'ner': 22.923011820075182}
Losses {'ner': 34.776931345162424}
Losses {'ner': 58.0865539298584}
Losses {'ner': 80.2331541045759}
Losses {'ner': 98.43938654589309}
Losses {'ner': 118.31708122290914}
Losses {'ner': 142.051808882672}
Losses {'ner': 168.30497145751733}
Losses {'ner': 196.89856779947533}
Losses {'ner': 201.4350271344005}
Losses {'ner': 207.8634976666295}
Losses {'ner': 215.5457424745976}
Losses {'ner': 227.24901329191187}
Losses {'ner': 240.1299841871049}
Losses {'ner': 254.5981495357795}
Losses {'ner': 273.3432953368184}
Losses {'ner': 294.22132534297134}
Losses {'ner': 316.0122356274883}
Losses {'ner': 340.91532929403957}
Losses {'ner': 365.5046948289023}
Losses {'ner': 391.5850121400076}
Losses {'ner': 421.95170790876773}
Losses {'ner': 455.8613114497243}
Losses {'ner': 489.67592513808313}
Losses {'ner': 492.4462929798724}
Losses {'ner': 497.0143177881514}
Losses {'ner': 502.65013106570814}
Losses {'ner': 510.3895629382355}
Losse

## Saving Model

In [21]:
nlp.to_disk("/content/Model")

In [22]:
# Save the  model to directory
output_dir = Path('/content/Model')
# Load the saved model and predict
print("Loading from", output_dir)
nlp_30 = spacy.load(output_dir)

for sentec, annotations in TRAIN_DATA_TEXT:
  # break
  if "React" in sentec:
    doc = nlp(sentec)
    print(sentec)
    print('***'*10)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Loading from /content/Model
JS, React, Node.js, JavaScript promises.
******************************
Entities [('React', 'skill'), ('Node.js', 'skill'), ('JavaScript', 'skill')]
JS, React, Node.js, JavaScript promises.
******************************
Entities [('React', 'skill'), ('Node.js', 'skill'), ('JavaScript', 'skill')]
